<a href="https://colab.research.google.com/github/MwasaGorret/UIPE-KATUNDA/blob/master/Object_detection_model_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Object Detection Model Tutorial
prerequisites:
1. A basic understanding of python programming and deep learning.

Check out this [tutorial](https://makmlclub.github.io/colab.html) to get familiar with Colab, and deep learning basics.

---
Notes on Object detection: </br>
[Object detection](https://en.wikipedia.org/wiki/Object_detection) is a machine learning technique under computer vision that involves locating the presence of objects with a bounding box and types or classes of the located objects in an image.
*   Input: An image with one or more objects, such as a photograph.
*   Output: One or more bounding boxes (e.g. defined by a point, width, and height), and a class label for each bounding box.




---

In this tutorial, we shall use the [Tensorflow Object Detection API](https://https://github.com/tensorflow/models/tree/master/research/object_detection), however,there are a number of object detection models in existence that you could use not to mention you can build one for yourself from scratch.

In [1]:
import os

In [2]:
!mkdir object_detection
os.chdir('object_detection/')

In [3]:
!git clone --depth=1 https://github.com/tensorflow/models.git

# TF-Slim is a lightweight library for defining, training and evaluating complex models in TensorFlow
!pip install tf_slim

Cloning into 'models'...
remote: Enumerating objects: 2305, done.
remote: Counting objects: 100% (2305/2305), done.
remote: Compressing objects: 100% (2000/2000), done.
remote: Total 2305 (delta 562), reused 953 (delta 282), pack-reused 0
Receiving objects: 100% (2305/2305), 30.60 MiB | 24.55 MiB/s, done.
Resolving deltas: 100% (562/562), done.
     |████████████████████████████████| 358kB 10.0MB/s 


## Protobuf installation/Compilation
Tensorflow Object Detection API uses Protobufs to configure model and training parameters. The Protobuf libraries must be installed to use the framework.

Notes on Protobufs:


---

*Protobufs*, or a serialized protocol buffer is a portable, extensible and efficient binary format developed by Google. Protobufs are defined by a simple language format (similar in spirit to a `C/C++ struct`) as defined below:
```
syntax: "proto3"
message Person {
  string name: 1;
  int32 id: 2;
  repeated string email:3;
}
```
This definition specifies the use of version 3 of `protobuf`, and specifies the each `Person` object may optionally have a `name` of type `string`, an `id` of `int32` and an `email` of type `string`. The numbers 1 to 3 are field identifiers used in each records binary representation. This definition can be saved in `.proto` file and compiled.

## Tensorflow Protobufs
Tensorflow primarily uses the `Example` protobuf, which represents a single instance in a dataset. It contains a list of named features, with each feature being either a list of byte strings, a list of floats, or a list of floats. An example is shown below:
```
syntax = "proto3";
message BytesList { repeated bytes value = 1; }
message FloatList { repeated float value = 1 [packed = true]; }
message Int64List { repeated int64 value = 1 [packed = true]; }
message Feature {
  oneof kind {
    BytesList bytes_list = 1;
    FloatList float_list = 2;
    Int64List int64_list = 3;
    }
};
message Features { map<string, Feature> feature = 1; };
message Example { Features features = 1; };
```
`[packed = true]` is used for repeated numerical fields to enusre efficient encoding.

---

Further reading:
1. [https://homl.info/protobuf](https://homl.info/protobuf)
2. Chapter 13: Hands-on Machine Learning with Scikit-Learn, Keras & TensorFlow by `Aurélien Géron`

In [4]:
%cd models/research
!protoc object_detection/protos/*.proto --python_out=.

pwd = os.getcwd()

# Add TF_slim to the system path.
os.environ['PYTHONPATH'] += f':{pwd}:{pwd}/slim'

/content/object_detection/models/research


## Adding necessary Environment Variables
Install the required packages from the `object_dection/models/research`.

In [5]:
!pwd

/content/object_detection/models/research


In [6]:
# Get the setup file
!cp object_detection/packages/tf2/setup.py .

# Installs from the setup.py file
!pip install .

Processing /content/object_detection/models/research
     |████████████████████████████████| 8.7MB 376kB/s 
     |████████████████████████████████| 849kB 19.5MB/s 
     |████████████████████████████████| 51kB 6.7MB/s 
     |████████████████████████████████| 61kB 9.0MB/s 
     |████████████████████████████████| 63.8MB 46kB/s 
     |████████████████████████████████| 829kB 51.7MB/s 
     |████████████████████████████████| 153kB 51.5MB/s 
     |████████████████████████████████| 2.0MB 50.5MB/s 
     |████████████████████████████████| 61kB 9.7MB/s 
     |████████████████████████████████| 174kB 40.4MB/s 
     |████████████████████████████████| 36.7MB 87kB/s 
     |████████████████████████████████| 102kB 13.2MB/s 
     |████████████████████████████████| 1.1MB 51.0MB/s 
     |████████████████████████████████| 112kB 26.1MB/s 
  Created wheel for object-detection: filename=object_detection-0.1-cp36-none-any.whl size=1599128 sha256=073d5b01426b52e1ba387f5b1069a909f84e39e96f89e5558601cf3feadfb829
 

In [7]:
# Test the setup
!python object_detection/builders/model_builder_tf2_test.py

2020-11-20 13:35:54.381880: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Running tests under Python 3.6.9: /usr/bin/python3
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_model
2020-11-20 13:35:57.104793: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2020-11-20 13:35:57.161699: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-11-20 13:35:57.162360: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.73GiB deviceMemoryBandwidth: 298.08GiB/s
2020-11-20 13:35:57.162410: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully o

Now the environment is setup and ready for use.

# Setting up the workspace
The pipeline to train an object detector involves a couple of steps that we shall go through one at a time.

1. Organize the workspace/training files
3. Generate tf records from the datasets
4. Configure a simple training pipeline
5. Train a model and monitor it’s progress
6. Export the resulting model and use it to detect objects.

In this tutorial, we use an already prepared and annotated dataset, the labelling was done using [Labelbox](https://labelbox.com) and the annotations extracted as a [JSON](https://www.tutorialspoint.com/json/index.htm) file. That step is skipped in this tutorial, however there is a number of open source resoures to used for data annotation including:
1. [LabelImg](https://github.com/tzutalin/labelImg)
2. [VGG Image Annotator](http://www.robots.ox.ac.uk/~vgg/software/via/)

At this stage, our directory should be as below;


```
object_detection
├─ models
│   ├─ community
│   ├─ research
│   └─ etc
```



## 1. Organise workspace/training files.
Create a folder under `object_detection` named `workspace` and under it create the `training` folder.

```
object_detection
├─ models
│   ├─ official
│   ├─ research
│   ├─ samples
│   └─ tutorials
└─ workspace
    └─ data
```
The data folder contains all the files related to training the model.

In [8]:
os.chdir('../../')
os.listdir()

['models']

In [9]:
!mkdir workspace
!mkdir workspace/data

The data folder structure is as defined below:
```
data
├─ annotations
├─ images
│   ├─ test
│   └─ train
├─ pre-trained-model
└─ training
```
Notes on training structure folder:

---

`annotations`: This folder stores all \*.csv files and the respective TensorFlow \*.record files, which contain the list of annotations for dataset images.

`images`: This folder contains a copy of all the images in our dataset.

`images\train`: This folder contains a copy of all images.

`images\test`: This folder contains a copy of all images.

`pre-trained-model`: This folder will contain the pre-trained model of your choice, which shall be used as a starting checkpoint for the training job.

`training`: This folder will contain the training pipeline configuration file \*.config, as well as a \*.pbtxt label map file and all files generated during the training of our model.

---

Before adding the data to the respective folders, we shall prepare it i.e. set it to the format compatible for use with `Tensorflow Object Detection API`.

In [10]:
!mkdir workspace/data/images 
!mkdir workspace/data/images/train
!mkdir workspace/data/images/test
!mkdir workspace/data/pre-trained-model
!mkdir workspace/data/training
!mkdir workspace/data/annotations

### Data Preparation
Upload the target dataset in a compressed format. The file should include the images and the annotations for each image in a unified JSON file.

```
dataset
├─ labels.json
└─ images
```

In [ ]:
# TODO
# 1. Upload the dataset file to the /object_detection directory.
# 2. Unzip the dataset file.

# NOTES: The dataset can be uploaded anywhere, as long as you keep track of the where.

In [11]:
!unzip /content/dataset.zip 

Archive:  /content/dataset.zip
   creating: dataset/
  inflating: dataset/labels.json     
   creating: dataset/images/
  inflating: dataset/images/img_117.jpg  
  inflating: dataset/images/img_423.jpg  
  inflating: dataset/images/img_363.jpg  
  inflating: dataset/images/img_475.jpg  
  inflating: dataset/images/img_55.jpg  
  inflating: dataset/images/img_448.jpg  
  inflating: dataset/images/img_176.jpg  
  inflating: dataset/images/img_140.jpg  
  inflating: dataset/images/IMG_20200205_114319_212.jpg  
  inflating: dataset/images/IMG_20200204_170753_308.jpg  
  inflating: dataset/images/img_85.jpg  
  inflating: dataset/images/img_543.jpg  
  inflating: dataset/images/img_66.jpg  
  inflating: dataset/images/img_73.jpg  
  inflating: dataset/images/IMG_20200205_155600_637.jpg  
  inflating: dataset/images/IMG_20200204_164024_837.jpg  
  inflating: dataset/images/img_364.jpg  
  inflating: dataset/images/IMG_20200205_114249_798.jpg  
  inflating: dataset/images/img_425.jpg  
  infl

## Data Preprocessing

Import modules for data preparation.

In [12]:
import json
import os
from pathlib import Path
import numpy as np
import shutil
import csv
import pandas as pd
from PIL import Image
from tqdm import tqdm

In [13]:
# Folder containing training images.
IMAGES_DIR = r'/content/object_detection/dataset/images' # Directory containing training images
LABELS_PATH = r'/content/object_detection/dataset/labels.json' # Path to the JSON labels file

### Utility Functions
To understand the bounding boxes and what each value in the bbox list represents, we draw from the COCO labeling format.

---

Labelbox creates a bounding box as follows:
```
bbox = ['xmin', 'ymin', 'width', 'height']
```
**`xmin` and `ymin` refer to the top left axis of the image.**

---

Further reading:
1. [https://www.immersivelimit.com/tutorials/create-coco-annotations-from-scratch](https://www.immersivelimit.com/tutorials/create-coco-annotations-from-scratch)

In [14]:
def json_to_csv_and_pbtxt(images_dir, labels_path, annotations_dir):
  """
  Converts a JSON file to a csv file and pbtxt file.

  TensorFlow requires a label map, which maps each of the used labels 
  to an integer values. This label map is used both by the training and detection 
  processes. Notice the labels are one-indexed i.e. start at 1 (one).

  Example:
  # example.pbtxt
  item {
    id: 1
    name: 'cat'
  }

  item {
    id: 2
    name: 'dog'
  }

  Both files are stored under the annotations folder.

  Note: This function is specific to labelbox labeling format.

  images_dir       is the directory containing the training images.
  labels_path      is the path to the labels JSON file.
  annotations_dir  is the directory in which the annotations will be stored.
  """
  # Stores the data from the JSON file.
  data_list = []

  # Open the labels.json from labels_dir
  with open(labels_path) as labels: 
        data_list.extend(json.load(labels)) 
      
  # Generating the csv in the annotations folder under data directory. (Ideally)
  csv_file_name = os.path.join(str(annotations_dir), 'labels.csv')

  with open(csv_file_name, 'w') as csv_label_file:
    f = csv.writer(csv_label_file)
    f.writerow(['file_name', 'width', 'height', 'class', 'xmin', 'ymin', 
                'xmax', 'ymax'])

    for item in tqdm(data_list, desc = "Images"):
        item_label = item['Label']

        # Check if the item['Label'] is empty. If empty, continue to the next item.
        if bool(item_label) == False:
            continue
        # Check if the item['Label']['objects'] is empty, if so, proceed to the next item.
        elif bool(item_label['objects'] == False):
            continue
        
        # Image file_name
        file_name = item['External ID']

        for bounding_box in item_label['objects']:
          class_name = bounding_box['title'].replace(" ", "")

          top = bounding_box['bbox']['top'] 
          left = bounding_box['bbox']['left']
          height = bounding_box['bbox']['height'] 
          width = bounding_box['bbox']['width']
                
          # Note that the axis refers to the top left corner of the image
          xmin = left 
          xmax = xmin + width
          ymin = top 
          ymax = ymin + height
          f.writerow([file_name, width, height, class_name, xmin, ymin, xmax, ymax])
                
  # Create the label map
  label_map_path = os.path.join(annotations_dir, "label_map.pbtxt")
  pbtxt_content = ""

  pbtxt_content = (
      pbtxt_content
      + "item {{\n    id: {0}\n    name: '{1}'\n}}\n\n".format(1, class_name)
  )
  pbtxt_content = pbtxt_content.strip()
  with open(label_map_path, "w") as f:
      f.write(pbtxt_content)              

In [ ]:
# TODO
# 1. Create the respective folders as defined above for the training directory.
# 2. Create the labels.csv and the labels_map.pbtxt files and store in the respective folders
#    using the function defined above.

# Note: Take care to use the correct directory and file paths.  

In [15]:
json_to_csv_and_pbtxt(IMAGES_DIR, LABELS_PATH, r'/content/object_detection/workspace/data/annotations')


Images: 100%|██████████| 581/581 [00:00<00:00, 117011.94it/s]


The `annotations` folder should now contain two files: `labels.csv' and 'label_map.pbtxt`

Split the `labels.csv` (remember it contains the entire training data) into train (0.8) and test set (0.2). For your expirements, create a validation set from the train set.

In [16]:
# The pandas library is used.
import pandas as pd

In [17]:
# NB: Ensure to set the ANNOTATIONS_DIR to the annotations directory.
dataset = pd.read_csv(os.path.join(r'/content/object_detection/workspace/data/annotations','labels.csv'))
dataset.head()

,file_name,width,height,class,xmin,ymin,xmax,ymax
0,img_17.jpg,54,84,brownspot,223,10,277,94
1,img_17.jpg,22,26,brownspot,369,312,391,338
2,img_562.jpg,56,70,brownspot,80,70,136,140
3,img_530.jpg,29,27,brownspot,130,275,159,302
4,img_498.jpg,67,37,brownspot,221,13,288,50


In [18]:
# Spliting the dataset into train (0.8) and test (0.2) sets respectively.
dataset_copy = dataset.copy()
train_set = dataset_copy.sample(frac=0.8, random_state=42)
test_set = dataset_copy.drop(train_set.index)

There are a couple of methods that can be used to split the dataset into train and test sets. A popular option is to use the  [sklearn train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html).

In [19]:
# Write the test_set and train_set to file.
train_set.to_csv(os.path.join(r'/content/object_detection/workspace/data/annotations', 'train_labels.csv'), index=False)
test_set.to_csv(os.path.join(r'/content/object_detection/workspace/data/annotations', 'test_labels.csv'), index=False)

Using the `train_set` and `test_set` we shall move the files to their respective folders in the data folder.

In [20]:
train_set.head()

,file_name,width,height,class,xmin,ymin,xmax,ymax
801,img_125.jpg,13,18,brownspot,96,352,109,370
677,img_447.jpg,42,42,brownspot,81,113,123,155
43,img_174.jpg,39,36,brownspot,149,241,188,277
990,img_420.jpg,102,139,brownspot,256,0,358,139
70,img_363.jpg,21,22,brownspot,111,98,132,120


In [21]:
# Move the train files to the train folder under the data directory.
# We use the copyfile function from the shutil library.
from shutil import copyfile

In [22]:
# Make sure the respective directories exist and or 
# keep track of your current directory.
# os.mkdirs(directory)

def move_files(files, source, dest):
  """Move files from the source directory to the destination directory."""
  for filename in files:
    copyfile(os.path.join(source, filename),
                 os.path.join(dest, filename))

TRAIN_DIR = r'workspace/data/images/train'
TEST_DIR = r'workspace/data/images/test'

TRAIN_IMAGES = train_set['file_name'].tolist()
TEST_IMAGES = test_set['file_name'].tolist()

# Move train files
move_files(TRAIN_IMAGES, IMAGES_DIR, TRAIN_DIR)
# Move test files
move_files(TEST_IMAGES, IMAGES_DIR, TEST_DIR)

Confirm that the images have been moved to the respective folders and clear the original folder to free up disk space.


```
# !rm -rf pathto/original_dataset
```



## 2. Generate TFRecords from the datasets.

In [23]:
# Support imports
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

import os
import io
import tensorflow as tf
import sys
sys.path.append("../models/research")

from PIL import Image
from object_detection.utils import dataset_util
from collections import namedtuple, OrderedDict

In [34]:
def class_text_to_int(row_label):
  if row_label == 'brown_spot': # the respective class_name
    return 1
  else:
    None

def split(df, group):
    data = namedtuple('data', ['file_name', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]

def create_tf_example(group, path):
    with tf.io.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    # check if the image format is matching with your images.
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(class_text_to_int(row['class']))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/file_name': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example

In [25]:
# TODO
# 1. Create TFRecords for both train.csv and test.csv.

labels = ['train_labels', 'test_labels']
paths = [TRAIN_DIR, TEST_DIR]

# Implement the creation of the *.record files using the functions defined above.

In [35]:
# Create tfrecords for train and test
for csv in ['train_labels']:
  writer = tf.io.TFRecordWriter('/content/object_detection/workspace/data/images/train/' + csv + '.record')
  path = r'/content/object_detection/dataset/images'
  examples = pd.read_csv( '/content/object_detection/workspace/data/annotations/train_labels.csv')
  grouped = split(examples, 'file_name')
  for group in grouped:
    tf_example = create_tf_example(group, path)
    writer.write(tf_example.SerializeToString())
    
  writer.close()
  print('Successfully created the TFRecords: {}'.format('/content/object_detection/workspace/data/images/train/' +csv + '.record'))


AttributeError: ignored

The `annotations` folder should now contain two more files: `test_labels.record` and `train_labels.record`

## Configure a simple training pipeline.
We use pretrained weights (randomly choosen from the [TensorFlow’s detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md)). For this tutorial, we shall use [faster_rcnn_inception_v2_coco_2018_01_28](http://download.tensorflow.org/models/object_detection/faster_rcnn_inception_v2_coco_2018_01_28.tar.gz).

---

Steps to follow:
1. Download the pretrained weights of your choice from [TensorFlow’s detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md).
2. Download the corresponding config file from [here](https://github.com/tensorflow/models/tree/master/research/object_detection/samples/configs) and save it under the `data/training` folder.
3. Extract the `*.tar.gz` file into the `pre-trained-model` folder under the `data` directory. 


In [ ]:
# Helper library
import urllib

In [ ]:
# 1. Download the pretrained weights of your choice from TensorFlow’s detection model zoo.
PRE_TRAINED_MODEL_DIR = r'workspace/data/pre-trained-model'
MODEL = 'faster_rcnn_inception_v2_coco_2018_01_28.tar.gz'

if MODEL not in os.listdir(PRE_TRAINED_MODEL_DIR):
  urllib.request.urlretrieve(f'http://download.tensorflow.org/models/object_detection/{MODEL}',\
                     os.path.join(PRE_TRAINED_MODEL_DIR, MODEL))
  !tar -zxvf {PRE_TRAINED_MODEL_DIR}/{MODEL} -C {PRE_TRAINED_MODEL_DIR} # Extract to a given directory

In [ ]:
# 2. Download the corresponding config file from here and save it under the data/training folder.
# config_file = 'faster_rcnn_inception_v2_coco.config'
# TRAINING_DIR = r'object_detection/workspace/data/training'

# if config_file not in os.listdir(TRAINING_DIR):
#   urllib.request.urlretrieve(f'https://github.com/tensorflow/models/blob/master/research/object_detection/samples/configs/{config_file}', \
#                              os.path.join(TRAINING_DIR, config_file))

# TODO: The code downloads an HTML file instead of the .config, requires further exploration.

Now open and edit the config file to fit your dataset. The config file also contains numerous hyperparameters that will be continously optimized for your model.

In [ ]:
# This is the .config file for faster_rcnn_incpetion_v2_coco.config, copied from
# https://github.com/tensorflow/models/tree/master/research/object_detection/samples/configs

%%writefile /content/object_detection/workspace/data/training/faster_rcnn_inception_v2_coco.config

# Open the file to see it's contents.
# Faster R-CNN with Inception v2, configuration for MSCOCO Dataset.
# Users should configure the fine_tune_checkpoint field in the train config as
# well as the label_map_path and input_path fields in the train_input_reader and
# eval_input_reader. Search for "PATH_TO_BE_CONFIGURED" to find the fields that
# should be configured.


model {
  faster_rcnn {
    num_classes: 1 # Change to the number of classes in your dataset.
    image_resizer {
      keep_aspect_ratio_resizer {
        min_dimension: 400
        max_dimension: 1024
      }
    }
    feature_extractor {
      type: 'faster_rcnn_inception_v2' # Change to the pretrained weights that you are using.
      first_stage_features_stride: 16
    }
    first_stage_anchor_generator {
      grid_anchor_generator {
        scales: [0.25, 0.5, 1.0, 2.0]
        aspect_ratios: [0.5, 1.0, 2.0]
        height_stride: 16
        width_stride: 16
      }
    }
    first_stage_box_predictor_conv_hyperparams {
      op: CONV
      regularizer {
        l2_regularizer {
          weight: 0.0
        }
      }
      initializer {
        truncated_normal_initializer {
          stddev: 0.01
        }
      }
    }
    first_stage_nms_score_threshold: 0.0
    first_stage_nms_iou_threshold: 0.7
    first_stage_max_proposals: 300
    first_stage_localization_loss_weight: 2.0
    first_stage_objectness_loss_weight: 1.0
    initial_crop_size: 14
    maxpool_kernel_size: 2
    maxpool_stride: 2
    second_stage_box_predictor {
      mask_rcnn_box_predictor {
        use_dropout: false
        dropout_keep_probability: 1.0
        fc_hyperparams {
          op: FC
          regularizer {
            l2_regularizer {
              weight: 0.0
            }
          }
          initializer {
            variance_scaling_initializer {
              factor: 1.0
              uniform: true
              mode: FAN_AVG
            }
          }
        }
      }
    }
    second_stage_post_processing {
      batch_non_max_suppression {
        score_threshold: 0.0
        iou_threshold: 0.6
        max_detections_per_class: 100
        max_total_detections: 300
      }
      score_converter: SOFTMAX
    }
    second_stage_localization_loss_weight: 2.0
    second_stage_classification_loss_weight: 1.0
  }
}

train_config: {
  batch_size: 4 # Increase/Decrease considering available memory.
  optimizer {
    momentum_optimizer: {
      learning_rate: {
        manual_step_learning_rate {
          initial_learning_rate: 0.0002
          schedule {
            step: 900000
            learning_rate: .00002
          }
          schedule {
            step: 1200000
            learning_rate: .000002
          }
        }
      }
      momentum_optimizer_value: 0.9
    }
    use_moving_average: false
  }
  gradient_clipping_by_norm: 10.0
  fine_tune_checkpoint: "/content/object_detection/workspace/data/pre-trained-model/faster_rcnn_inception_v2_coco_2018_01_28/model.ckpt" # Replace with your pretrained weights.
  from_detection_checkpoint: true
  # Note: The below line limits the training process to 200K steps, which we
  # empirically found to be sufficient enough to train the COCO dataset. This
  # effectively bypasses the learning rate schedule (the learning rate will
  # never decay). Remove the below line to train indefinitely.
  num_steps: 200000
  data_augmentation_options {
    random_horizontal_flip {
    }
  }
}

train_input_reader: {
  tf_record_input_reader {
    input_path: "/content/object_detection/workspace/data/annotations/train_labels.record" # Path to training TFRecord
  }
  label_map_path: "/content/object_detection/workspace/data/annotations/label_map.pbtxt"
}

eval_config: {
  # num_examples: 8000
  # Note: The below line limits the evaluation process to 10 evaluations.
  # Remove the below line to evaluate indefinitely.
  # max_evals: 10
}

eval_input_reader: {
  tf_record_input_reader {
    input_path: "/content/object_detection/workspace/data/annotations/test_labels.record"
  }
  label_map_path: "/content/object_detection/workspace/data/annotations/label_map.pbtxt"
  shuffle: false
  num_readers: 1
}

## Monitoring training progress.
Tensorflow provides the [Tensorboard](https://www.tensorflow.org/programmers_guide/summaries_and_tensorboard) utility library that allows you to continuously monitor and visualize a number of different training/evaluation metrics. 

---
If you run Tensorboard before training the model, then you are able to monitor progress concurrently.

In [ ]:
# 1. Run the command below to run tensorboard, --logdir should point to the folder that stores the 
#    checkpoints during training.
%load_ext tensorboard
%tensorboard --logdir=workspace/data/training/ --port=6006

## Training a model
For training, copy the `object_detection/models/research/object_detection/model_main.py` file to `object_detection/workspace/data` folder.

---



In [ ]:
# 1. Copy the model_main.py into the object_detection/workspace/data
copyfile('models/research/object_detection/model_main_tf2.py',\
         'workspace/data/model_main_tf2.py')

In [ ]:
# 3. Run model_main.py script to start training, take note of the different arguments
#    supplied to the script and update accordingly.
!python workspace/data/model_main_tf2.py \
        --model_dir=training/ \
        --pipeline_config_path=training/faster_rcnn_inception_v2_coco.config

To monitor progress, refer to the tensorboard utility running in the cell prior to this one.

## Export the resulting model and use it to detect objects.
In most cases, extracting a trained inference graph is necessary, as it can be used to perform object detection in deployment.

---


Steps:
1. Copy the `models/research/object_detection/exporter_main_v2.py` script and paste it straight into the `object_detection/workspace/data` folder.

2. Inside the `data/training` folder, find the ckpt-* checkpoint file with the highest number following the name of the dash e.g. model.ckpt-34350). This number represents the training step index at which the file was created.

3. Make a note of the file’s name, as it will be passed as an argument when we call the `export_inference_graph.py` script.

In [ ]:
# Copy the exporter script
copyfile('models/research/object_detection/exporter_main_v2.py',\
         'workspace/data/exporter_main_v2.py')

In [ ]:
# cd inside your data/ folder, and run the following command:
!python workspace/data/exporter_main_v2.py \
    --input_type image_tensor \
    --pipeline_config_path training/faster_rcnn_inception_v2_coco.config \
    --trained_checkpoint_prefix training/model.ckpt-13302 
    --output_directory trained-inference-graphs/output_inference_graph_v1.pb


# References:
1. [https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/training.html](https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/training.html)